In [16]:
import pandas as pd
import numpy as np

In [17]:
folder = '/Users/jwang72/git/2018_works/firstdonoharm/hw10'
df_annotation = pd.read_csv(f'{folder}/annotation_dump_20190810.tsv', sep='\t')
df_annotation.head()


,keyId,sentId,annotator,label,time
0,abuta_1_1-Jun,abuta_1_1,Jun,not side effect,2019-06-02 21:26:11
1,abuta_1_1-Kelly,abuta_1_1,Kelly,not side effect,2019-06-02 21:44:39
2,abuta_1_2-Jun,abuta_1_2,Jun,not side effect,2019-06-02 21:26:16
3,abuta_1_2-Kelly,abuta_1_2,Kelly,not side effect,2019-06-02 21:44:39
4,abuta_2_0-Jun,abuta_2_0,Jun,not side effect,2019-06-02 21:26:19


In [18]:
df_sentence = pd.read_csv(f'{folder}/sentences.csv')
df_sentence.head()

,sentId,sentence
0,abuta_1_1,I did not realize until Monday night 6/24 that...
1,abuta_1_2,It is the only new supplement I had started re...
2,abuta_2_0,I started taking Royal Abuta Plus on the 20th ...
3,abuta_2_1,"Since then, I have had 4 carbuncles in a perio..."
4,abuta_2_2,I don't know if it related but it sure seem co...


In [19]:
df = df_sentence.merge(df_annotation[df_annotation.annotator=='Kelly']['sentId label'.split()], on='sentId')
df.head()

,sentId,sentence,label
0,abuta_1_1,I did not realize until Monday night 6/24 that...,not side effect
1,abuta_1_2,It is the only new supplement I had started re...,not side effect
2,abuta_2_0,I started taking Royal Abuta Plus on the 20th ...,not side effect
3,abuta_2_1,"Since then, I have had 4 carbuncles in a perio...",side effect
4,abuta_2_2,I don't know if it related but it sure seem co...,not side effect


In [20]:
df.label.value_counts()

not side effect    770
side effect         78
skip                 1
Name: label, dtype: int64

In [21]:
df = df[df.label.isin(['not side effect', 'side effect'])]
df.label.value_counts()

not side effect    770
side effect         78
Name: label, dtype: int64

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import LinearSVC, SVC 
from sklearn.model_selection import train_test_split

In [23]:
vec = CountVectorizer(ngram_range=(1, 2), min_df=2)
X = vec.fit_transform(df['sentence']).astype(float)
y = df['label']

In [24]:
vec.vocabulary_

{'did': 440,
 'not': 1083,
 'realize': 1254,
 'until': 1670,
 'night': 1066,
 '24': 16,
 'that': 1480,
 'it': 833,
 'was': 1716,
 'royal': 1297,
 'abuta': 41,
 'plus': 1206,
 'causing': 348,
 'the': 1493,
 'problem': 1228,
 'did not': 441,
 'that it': 1485,
 'it was': 852,
 'royal abuta': 1298,
 'abuta plus': 42,
 'is': 816,
 'only': 1147,
 'new': 1063,
 'supplement': 1430,
 'had': 672,
 'started': 1392,
 'recently': 1260,
 'it is': 847,
 'is the': 825,
 'the only': 1518,
 'had started': 676,
 'taking': 1459,
 'on': 1137,
 'of': 1114,
 'started taking': 1393,
 'on the': 1140,
 'since': 1349,
 'then': 1540,
 'have': 693,
 'in': 785,
 'period': 1196,
 'months': 988,
 'have had': 698,
 'of months': 1120,
 'don': 471,
 'know': 874,
 'if': 773,
 'related': 1278,
 'but': 312,
 'sure': 1434,
 'seem': 1323,
 'don know': 472,
 'know if': 875,
 'if it': 774,
 'but it': 314,
 'twice': 1656,
 'day': 421,
 'half': 681,
 'tablespoon': 1448,
 'each': 499,
 'time': 1587,
 'twice day': 1657,
 'half of'

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

print(y_train.value_counts())
print(y_test.value_counts())

not side effect    689
side effect         74
Name: label, dtype: int64
not side effect    81
side effect         4
Name: label, dtype: int64


In [26]:
model = LinearSVC(C=1, class_weight='balanced', random_state=0)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [27]:
topk = np.argsort(np.abs(model.coef_.reshape(-1)))[::-1][:30]
topk

array([1196, 1520,  707,  799, 1246, 1808,  572, 1150, 1372,  712,  541,
        713,  816,  272,  421,  839,  453, 1473, 1415, 1731,  608,  756,
         51,  925, 1722,  524,  130,  546, 1556,  148])

In [28]:
for w, i in vec.vocabulary_.items():
    if i in topk:
        print(w)

is
period
day
worse
having
episodes
between
the pain
was going
and it
it did
felt
experienced
some
hot
water
for my
rash
headaches
disappeared
only side
terrible
and then
increase
added
lost
things
headache
strong
except


In [29]:
np.mean(y_pred == y_test)

0.8941176470588236